In [1]:
from bs4 import BeautifulSoup
from requests import get
import re
from datetime import datetime

In [2]:
APARTMENT = 'Ava Pacific Beach'

url = 'https://api.avalonbay.com/json/reply/ApartmentSearch?communityCode=CA025&min=1600&max=2800&_=1550548620970'
r = get(url, timeout=5).json()

available_floorplans = r['results']['availableFloorPlanTypes']
two_bedroom = [x for x in available_floorplans if x['floorPlanTypeCode'] == '2BD'][0]['availableFloorPlans']

In [3]:
two_bedroom[0]['finishPackages'][0]['apartments'][0]

{'apartmentCode': 'CA025-CA025-00P-P303',
 'communityCode': 'CA025',
 'subCommunityCode': None,
 'buildingNumber': '00P',
 'apartmentNumber': '00P-P303',
 'apartmentAddress': '3952 Jewell Street P303',
 'floorPlan': None,
 'pricing': {'amenitizedRent': 2510.0,
  'effectiveRent': 2510.0,
  'availableDate': '/Date(1552867200000)/',
  'showAsAvailable': True,
  'unavailableMessage': None,
  'leaseTerms': None},
 'hasPromotion': False,
 'promotionList': [],
 'amenityList': None,
 'finishPackage': None,
 'apartmentSize': 988,
 'beds': 2,
 'baths': 2.0,
 'onlineApplicationUrl': '',
 'unitKey': 'CA025-CA025-00P-P303',
 'floor': 3}

In [4]:
listings = []

for fp in two_bedroom:

    for a in fp['finishPackages'][0]['apartments']:

        data = {}

        data['floorplan'] = 'TODO'
        data['unit'] = a['apartmentNumber']
        data['floor'] = a['floor']
        data['beds'] = a['beds']
        data['baths'] = a['baths']

        # available date
        epoch_str = re.search('/Date\((\d+)\)/', a['pricing']['availableDate']).group(1)
        data['available_dt'] = datetime.fromtimestamp(int(epoch_str) / 1000.0).strftime('%Y-%m-%d')
        
        # go to subpage to get more details
        apartment_code = a['apartmentCode']
        url = 'https://www.avaloncommunities.com/california/san-diego-apartments/ava-pacific-beach/apartment/' + apartment_code

        r2 = get(url, timeout=20)
        soup = BeautifulSoup(r2.content, "html.parser")
        
        description_el = soup.find('div', {'class': 'description'})
        data['size'] = int(re.search(r'(\d+) sq ft', description_el.text).group(1))

        # terms
        leases = soup.find('div', {'class': 'terms'}).select('table')[0].findAll('tr')
        data['terms'] = []
        for lease in leases:
            
            td_elements = lease.select('td')
            
            length = re.sub('[^0-9]', '', td_elements[0].text)  # regex to remove all non-digits
            price = int(re.sub('[^0-9]', '',  td_elements[-1].text)) # regex to remove all non-digits

            if int(length) > 5:
                data['terms'].append({'k': length, 'v': price})


        listings.append(data)


listings

[{'floorplan': 'TODO',
  'unit': '00P-P303',
  'floor': 3,
  'beds': 2,
  'baths': 2.0,
  'available_dt': '2019-03-18',
  'size': 988,
  'terms': [{'k': '12', 'v': 2510},
   {'k': '11', 'v': 2715},
   {'k': '10', 'v': 2720},
   {'k': '9', 'v': 3935},
   {'k': '8', 'v': 3960},
   {'k': '7', 'v': 3900},
   {'k': '6', 'v': 4135}]},
 {'floorplan': 'TODO',
  'unit': '00V-V206',
  'floor': 2,
  'beds': 2,
  'baths': 2.0,
  'available_dt': '2019-03-18',
  'size': 988,
  'terms': [{'k': '12', 'v': 2565},
   {'k': '11', 'v': 2770},
   {'k': '10', 'v': 2775},
   {'k': '9', 'v': 3990},
   {'k': '8', 'v': 4015},
   {'k': '7', 'v': 3955},
   {'k': '6', 'v': 4190}]},
 {'floorplan': 'TODO',
  'unit': '00V-V106',
  'floor': 1,
  'beds': 2,
  'baths': 2.0,
  'available_dt': '2019-03-18',
  'size': 988,
  'terms': [{'k': '12', 'v': 2575},
   {'k': '11', 'v': 2780},
   {'k': '10', 'v': 2785},
   {'k': '9', 'v': 4000},
   {'k': '8', 'v': 4025},
   {'k': '7', 'v': 3965},
   {'k': '6', 'v': 4200}]},
 {'floo